In [ ]:
from qubitdriver import qubit

from matplotlib import pyplot as plt
import numpy as np

from qsystem_2 import *

In [ ]:
q = qubit()

In [ ]:
pulseCount = 9
nsamp = 100
nsync = int(1.1 * nsamp)

idec,qdec,iacc,qacc = q.runPulseExample(pulseCount = pulseCount, nsamp = nsamp, nsync = nsync)

In [ ]:
amplitudes = np.abs(idec + 1j*qdec)
plt.plot(amplitudes)
for i in range(pulseCount+1):
    plt.axvline(i*nsamp, c='r')
    if i < pulseCount:
        plt.axvline((i+0.5)*nsamp, c='g', alpha=0.5)

In [ ]:
iaccSums = idec.reshape(-1, nsamp).sum(axis=1)
qaccSums = qdec.reshape(-1, nsamp).sum(axis=1)

di_feed = q.get_i_acc()
dq_feed = q.get_q_acc()
if di_feed>2**31:
    di_feed = di_feed - 2**32
if dq_feed>2**31:
    dq_feed = dq_feed - 2**32

fig,ax = plt.subplots(4,1,sharex=True)

fig.suptitle("Before Moving Average Results")
fig.set_size_inches(8,8)
ax[0].plot(iaccSums, "-+", label="get_decimated")
ax[0].plot(iacc, "x", label="get_accumulated")
ax[0].plot(len(iacc)-1, di_feed, "o", mfc='None', label='feedback')
ax[0].set_ylabel("I")
ax[0].legend(bbox_to_anchor=(0, 1, 1, 0), loc="lower left", mode="expand", ncol=3)

ax[1].plot(qaccSums, "-+")
ax[1].plot(qacc, "x")
ax[1].plot(len(iacc)-1, dq_feed, "o", mfc='None', label='feedback')
ax[1].set_ylabel("Q")

ax[2].plot(np.abs(iaccSums + 1j*qaccSums), "-+")
ax[2].plot(np.abs(iacc + 1j*qacc), "x")
ax[2].plot(len(iacc)-1, np.abs(di_feed + 1j*dq_feed), "o", mfc='None', label='feedback')
ax[2].set_ylabel("Amplitude")

ax[3].plot(np.arctan2(qaccSums, iaccSums), "-+")
ax[3].plot(np.arctan2(qacc, iacc), "x")
ax[3].plot(len(iacc)-1, np.arctan2(dq_feed, di_feed), "o", mfc='None', label='feedback')
ax[3].set_ylabel("$\phi$ [Rad]")

ax[3].set_xlabel("Pulse Number")

In [ ]:
#Divide the amplitude array into segments and create the gaussian curve that matches the data. 

experiments = amplitudes.reshape(-1, nsamp)
gaussian = gauss(mu=16*nsamp/2, si=nsamp, length=16*nsamp, maxv=30000)
gaussian = gaussian[0::16]

experiments = np.array(experiments)
gaussian = np.array(gaussian)

In [ ]:
#Check the alignment of the gaussian with the first experimental data

print(np.amax(experiments[0]))
print(np.amax(gaussian))

print(np.where(experiments[0] == np.amax(experiments[0]))[0][0])
print(np.where(gaussian == np.amax(gaussian))[0][0])

plt.plot(gaussian * 0.0215, alpha=0.5)
plt.plot(experiments[0])
plt.axvline(np.where(experiments[0] == np.amax(experiments[0]))[0][0], color='blue', alpha=0.5)
plt.axvline(np.where(gaussian == np.amax(gaussian))[0][0], color='orange', alpha=0.5)
plt.show()

In [ ]:
#Find the weighted average of the data. 
newAmps = []
for i in range(len(experiments)):
    experiments[i] = experiments[i] * gaussian
    newAmps.append(np.sum(experiments[i]) / np.sum(gaussian))

In [ ]:
#Plot the results:

plt.plot(newAmps)
plt.title("Amplitude After Weighted Average")
plt.show()

oldAmps = np.abs(iaccSums + 1j*qaccSums)
print("Range before weighting: ", np.amax(oldAmps) - np.amin(oldAmps))
print("Range after weighting: ", np.amax(newAmps) - np.amin(newAmps))